In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.activations import selu
from keras.optimizers import Adam, Adadelta, Adagrad, RMSprop
from keras.layers import Dense
from keras.layers import Dropout
import keras.backend as K
import gzip

Using TensorFlow backend.


In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
##meta_data = getDF('meta_Electronics.json.gz')

In [4]:
data = pd.read_csv('ratings_Electronics.csv', header=None)

In [ ]:
##meta_data = meta_data.drop(['imUrl', 'salesRank', 'related', 'brand'],axis=1)
##meta_data.head(10)

In [5]:
#data = data.iloc[1:]
data = data.rename(columns={0: "reviewerID", 1: "asin", 2: "overall", 3:"reviewTime"})
data = data.drop('reviewTime',axis=1)
data.head(10)

,reviewerID,asin,overall
0,AKM1MP6P0OYPR,0132793040,5.0
1,A2CX7LUOHB2NDG,0321732944,5.0
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0
5,A1QGNMC6O1VW39,0511189877,5.0
6,A3J3BRHTDRFJ2G,0511189877,2.0
7,A2TY0BTJOTENPG,0511189877,5.0
8,A34ATBPOK6HCHY,0511189877,5.0
9,A89DO69P0XZ27,0511189877,5.0


In [6]:
data.shape

(7824482, 3)

In [7]:
# cast the rating to float32 to save memory 
data['overall'] = data['overall'].astype('float32')

# cut down item count by limiting the entry of the dataset, only include them when it has more than 20 ratings
item_rate = data['asin'].value_counts()
print(item_rate.head(10))
item_rate1 = pd.DataFrame(data=item_rate)
item_rate1.columns = ['rating_count']
rate_count = 20
item_rate1.sort_values(by=['rating_count'])
item_rate1 = item_rate1.loc[item_rate1['rating_count'] >= rate_count]
item_list = item_rate1.index.values
data = data.loc[data['asin'].isin(item_list)]

B0074BW614    18244
B00DR0PDNE    16454
B007WTAJTO    14172
B0019EHU8G    12285
B006GWO5WK    12226
B003ELYQGG    11617
B003ES5ZUU    10276
B007R5YDYA     9907
B00622AG6S     9823
B0002L5R78     9487
Name: asin, dtype: int64


In [8]:
data.shape

(6341872, 3)

In [9]:
user_rate = data['reviewerID'].value_counts()
print(user_rate.head(10))
user_rate1 = pd.DataFrame(data=user_rate)
user_rate1.columns = ['rating_count']
rate_count = 15
user_rate1.sort_values(by=['rating_count'])
user_rate1 = user_rate1.loc[user_rate1['rating_count'] >= rate_count]
user_list = user_rate1.index.values
data = data.loc[data['reviewerID'].isin(user_list)]

ADLVFFE4VBT8      395
A3OXHLG6DIBRW8    375
A6FIAB28IS79      352
A5JLAU2ARJ0BO     329
A680RUE1FDO8B     325
A1ODOGXEYECQQ8    301
A36K2N527TXXJN    253
ARBKYIVNYWK3C     239
AWPODHOB4GFWL     230
A22CW0ZHY3NJH8    225
Name: reviewerID, dtype: int64


In [10]:
# remove to free up RAM 
del user_rate
del user_rate1
del user_list
del item_rate
del item_rate1
del item_list

In [12]:
curr_prod_list = data['asin'].unique().tolist()
#data = data.loc[data['reviewerID'].isin(user_list)]
print(len(curr_prod_list))
##print(meta_data.shape)
##meta_data = meta_data.loc[meta_data['asin'].isin(curr_prod_list)]
##print(meta_data.shape)

48155


In [13]:
print(data.shape)
print(data['reviewerID'].nunique())
print(data['asin'].nunique())
data.head()

(412410, 3)
16389
48155


,reviewerID,asin,overall
17,A1H8PY3QHMQQA0,0528881469,2.0
113,A2QBZA4S1ROX9Q,0594481813,3.0
118,AT09WGFUM934H,0594481813,3.0
189,A2IDCSC6NVONIZ,0972683275,5.0
200,A3BMUBUC1N77U8,0972683275,4.0


In [ ]:
##meta_data = meta_data.reset_index()
##meta_data = meta_data.drop(['index'],axis=1)
##meta_data.head(5)

In [14]:
duplicateRowsDF = data[data.duplicated(['reviewerID', 'asin'])]
print("Duplicate Rows based on 2 columns are:", duplicateRowsDF, sep='\n')

Duplicate Rows based on 2 columns are:
Empty DataFrame
Columns: [reviewerID, asin, overall]
Index: []


In [15]:
# train and test split 
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data,
                                    stratify=data['reviewerID'],
                                    test_size=0.2,
                                    random_state = 225)

In [16]:
#user_item_mtx_train = matrix_conversion(train_df)
mtx_data = data.pivot(index='reviewerID', columns='asin', values='overall').fillna(0)
#user_item_mtx_test = test_df.pivot(index='reviewerID', columns='asin', values='overall').fillna(0)

In [17]:
del data
mtx_data.head(5)

asin,0528881469,0594481813,0972683275,0979526191,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,...,B00KJJW36G,B00KJLZDXO,B00KMRGF28,B00KNM763E,B00KPRWAX8,B00KSLCU72,B00KWHMR6G,B00KYMCJF8,B00L26YDA4,B00L3YHF6O
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0655860XQH1M5Q8JH1M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1004703RC79J9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10063PJ5C9WQQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100NGGXRQF0AQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100UD67AHFODS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# this is to ensure the train and test size stays the same while only contains test/train rating data
def Preprocess_Data(df, df_to_convert, init_value=0, method="None"):
    """
        INPUT: 
            df = original pivot table that contains all the users and items
            df_to_convert = train/test dataset to convert into pivot table
            init_value = initial value when creating the dataset
            
        OUTPUT:
            pviot table matrix with user as row and item as column. 
    """
    # copy the user item matrix 
    user_item_mtx = df.copy(deep=True)
    # set initialize value
    user_item_mtx.loc[:,:] = init_value
    
    if method == "Mean":
        # mean normalization, subtract all ratings by the products average ratings
        # test required before training
        for (_, reviewerID, asin, rating) in df_to_convert.itertuples():
            user_item_mtx.loc[reviewerID, asin] = float(rating)
        # subtract mean of each product
        user_item_mtx = user_item_mtx - user_item_mtx.mean()
        
    elif method == "None":
        for (_, reviewerID, asin, rating) in df_to_convert.itertuples():
            user_item_mtx.loc[reviewerID, asin] = float(rating)

    user_list = df_to_convert['reviewerID'].unique().tolist()
    user_item_mtx = user_item_mtx.ix[user_list]
    
    return user_item_mtx

In [19]:
usr_itm_mtx_train = Preprocess_Data(mtx_data, train_df, 0, method="None")
usr_itm_mtx_test = Preprocess_Data(mtx_data, test_df, 0, method="None")

In [20]:
usr_itm_mtx_train

asin,0528881469,0594481813,0972683275,0979526191,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,...,B00KJJW36G,B00KJLZDXO,B00KMRGF28,B00KNM763E,B00KPRWAX8,B00KSLCU72,B00KWHMR6G,B00KYMCJF8,B00L26YDA4,B00L3YHF6O
reviewerID,,,,,,,,,,,,,,,,,,,,,
A3CFU6MWOKDVP4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A3VVLD0I7AITYN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2UX1QYTF609V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2JKM8KF3296CW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1HF7HD4IO7ZJI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1JOCFRO3UC0QE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A3AQO91DYWF8R0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A141R97FHSSUQ0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A275L12XC23G9I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
usr_itm_mtx_train.shape

(16389, 48155)

In [22]:
usr_itm_mtx_test.shape

(16389, 48155)

In [23]:
def masked_se(yTrue, yPred):
        # masked function
        mask_true = K.cast(K.not_equal(yTrue, 0), K.floatx())
        # masked squared error
        masked_squared_error = K.square(mask_true * (yTrue - yPred))
        masked_mse = K.sum(masked_squared_error, axis=-1)/K.sum(mask_true, axis=-1)
        return masked_mse

def customLoss(yTrue,yPred):
    yNotZero = K.cast(yTrue!=0, 'float32')
    return K.sum(yNotZero*(K.square(yTrue-yPred)))/K.sum(yNotZero)

In [24]:
def customRMSE(yTrue, yPred):
    yNotZero = K.cast(yTrue!=0, 'float32')
    return K.sqrt(customLoss(yTrue, yPred))

def customRMSE_withClip(yTrue, yPred):
    yNotZero = K.cast(yTrue!=0, 'float32')
    # scale the current values back to 1 and 5
    y_pred = K.clip(yPred, 1, 5)
    
    return K.sqrt(customLoss(yTrue, yPred))

def masked_rmse_clip(yTrue, yPred):
    # masked function
    #mask_true = K.cast(K.not_equal(yTrue, 0), K.floatx())
    mask_true = K.cast(yTrue!=0, 'float32')
    yPred = K.clip(yPred, 1, 5)
    #yPred = np.clip(yPred, a_min =1, a_max = 5)
    # masked squared error
    masked_squared_error = K.square(mask_true * (yTrue - yPred))
    masked_mse = K.sqrt(K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1))
        return masked_mse

In [34]:
from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import regularizers

# input and activation 
def autoEncoder(X, act):
    '''
    Autoencoder for Collaborative Filter Model
    '''
    # Input
    input_layer = Input(shape=(X.shape[1],), name='UserScore')
    
    # Encoder
    # -----------------------------
    enc = Dense(256, activation=act, name='EncLayer1')(input_layer)

    # Latent Space
    # -----------------------------
    lat_space = Dense(128, activation=act, name='LatentSpace')(enc)
    lat_space = Dropout(0.8, name='Dropout')(lat_space) # Dropout

    # Decoder
    # -----------------------------
    dec1 = Dense(128, activation=act, name='DecLayer1')(lat_space)
    dec2 = Dense(256, activation=act, name='DecLayer2')(dec1)

    # Output
    #output_layer = Dense(X.shape[1], activation=act, name='UserScorePred')(dec2)
    output_layer = Dense(X.shape[1], name='UserScorePred')(dec2)
    
    # Complete the model by adding input and output layer
    model = Model(input_layer, output_layer)    
    
    return model

In [35]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [36]:
# Build model
model = autoEncoder(usr_itm_mtx_train.values, 'selu')

#model.compile(optimizer = Adam(lr=0.0001),loss=customLoss, metrics=[customRMSE_withClip])
model.compile(optimizer = Adam(lr=0.001),loss=customLoss, metrics=[masked_rmse_clip])

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
UserScore (InputLayer)       (None, 48155)             0         
_________________________________________________________________
EncLayer1 (Dense)            (None, 256)               12327936  
_________________________________________________________________
LatentSpace (Dense)          (None, 128)               32896     
_________________________________________________________________
Dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
DecLayer1 (Dense)            (None, 128)               16512     
_________________________________________________________________
DecLayer2 (Dense)            (None, 256)               33024     
_________________________________________________________________
UserScorePred (Dense)        (None, 48155)             1237

In [37]:
hist = model.fit(x=usr_itm_mtx_train.values, 
                  y=usr_itm_mtx_train.values,
                  epochs=20,
                  validation_split=0.2)

Train on 13111 samples, validate on 3278 samples
Epoch 1/20
13111/13111 [==============================] - 103s 8ms/step - loss: 5.6623 - masked_rmse_clip: 1.9128 - val_loss: 2.9310 - val_masked_rmse_clip: 1.4925
Epoch 2/20
13111/13111 [==============================] - 103s 8ms/step - loss: 2.2362 - masked_rmse_clip: 1.3129 - val_loss: 3.4230 - val_masked_rmse_clip: 1.7005
Epoch 3/20
13111/13111 [==============================] - 105s 8ms/step - loss: 1.6784 - masked_rmse_clip: 1.1951 - val_loss: 3.8329 - val_masked_rmse_clip: 1.8306
Epoch 4/20
13111/13111 [==============================] - 102s 8ms/step - loss: 1.5226 - masked_rmse_clip: 1.1422 - val_loss: 3.3501 - val_masked_rmse_clip: 1.6843
Epoch 5/20
13111/13111 [==============================] - 103s 8ms/step - loss: 1.3763 - masked_rmse_clip: 1.0877 - val_loss: 2.7193 - val_masked_rmse_clip: 1.4849
Epoch 6/20
13111/13111 [==============================] - 101s 8ms/step - loss: 1.2943 - masked_rmse_clip: 1.0493 - val_loss: 3.017

In [38]:
pred = model.predict(usr_itm_mtx_test.values)

In [ ]:
def masked_RMSE_function(yTrue, yPred):
    # masked function
    mask_true = K.cast(K.not_equal(yTrue, 0), K.floatx())
    #mask_true = np.where(yTrue != 0, 1, 0)
    #mask_true = yTrue!=0, 'float32')
    yPred = np.clip(yPred, 1, 5)
    # masked squared error
    masked_squared_error = (mask_true * (yTrue - yPred))**2
    # np.sum(masked_squared_error, axis=-1)
    masked_mse = np.sqrt( np.sum(masked_squared_error) / np.sum(mask_true))

    return masked_mse

RMSE_rest = masked_RMSE_function(usr_itm_mtx_test.values, pred)
RMSE_rest